# Image Search

## Library

In [ ]:
%%capture
!pip install sentence_transformers -qq
!pip install -U gdown -qq
!pip install annoy -qq

In [ ]:
import shutil
import gdown
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

import cv2
from google.colab.patches import cv2_imshow

from PIL import Image, ImageOps

import torch
import torchvision.transforms as T

import matplotlib.pyplot as plt

from natsort import natsorted

from sklearn.metrics import f1_score

#scipy
from scipy import stats #ensemble

#sentence_transformers
from sentence_transformers import SentenceTransformer

#Annoy index
from annoy import AnnoyIndex

## Load Data

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c appman-image-search

 96% 33.0M/34.5M [00:02<00:00, 17.7MB/s]
100% 34.5M/34.5M [00:02<00:00, 14.1MB/s]


In [ ]:
!unzip -q /content/appman-image-search.zip

## Preprocessing

### Manage Folder

สร้าง folder สำหรับแต่ละ class ขึ้นมา เพราะตอนแรกไฟล์มันมาเดี่ยวๆ แล้วมันจัดการยาก

In [ ]:
#list of image
lis = glob('/content/queries/queries/*.jpg')
#sort by class
lis = natsorted(lis)
lis[:10]

['/content/queries/queries/0.jpg',
 '/content/queries/queries/1.jpg',
 '/content/queries/queries/2.jpg',
 '/content/queries/queries/3.jpg',
 '/content/queries/queries/4.jpg',
 '/content/queries/queries/5.jpg',
 '/content/queries/queries/6.jpg',
 '/content/queries/queries/7.jpg',
 '/content/queries/queries/8.jpg',
 '/content/queries/queries/9.jpg']

In [ ]:
#move file to folder of their class
for cls, src in enumerate(lis):
  #create folder
  folder = f'/content/queries/queries/{cls}'
  %mkdir $folder
  #-----------------

  #source and destination
  base_name = os.path.basename(src)
  des = os.path.join(folder, base_name)
  #-------------------

  #move file
  os.rename(src, des)
  #------------

### Augmentation

In [ ]:
#list of image
lis = glob('/content/queries/queries/**/*.jpg')
#sort by class
lis = natsorted(lis)
lis[:3]

['/content/queries/queries/0/0.jpg',
 '/content/queries/queries/1/1.jpg',
 '/content/queries/queries/2/2.jpg']

In [ ]:
for img_path in tqdm(lis):
  #load image
  img = Image.open(img_path).convert('RGB')
  #---------

  #rotate image
  for i in range(15):
    #degree
    degree = i+1

    #image rotate plus degree
    img_rotate = T.functional.rotate(img, angle=degree, fill=255)
    save_path = img_path[:-4]
    save_path = f'{save_path}_plus_{i}.jpg'
    img_rotate.save(save_path)
    #---------

    #image rotate negative degree
    img_rotate = T.functional.rotate(img, angle=-degree, fill=255)
    save_path = img_path[:-4]
    save_path = f'{save_path}_negative_{i}.jpg'
    img_rotate.save(save_path)
    #---------

  #------------------------------------

100%|██████████| 22/22 [00:02<00:00,  9.78it/s]


### Resize image with padding

In [ ]:
#list of image
lis = glob('/content/queries/queries/**/*.jpg')
#sort by class
lis = natsorted(lis)
lis[:3]

['/content/queries/queries/0/0.jpg',
 '/content/queries/queries/0/0_negative_0.jpg',
 '/content/queries/queries/0/0_negative_1.jpg']

In [ ]:
#function for resize and add pading for keep aspect ratio
def resize_pad(img_path):
  #load image
  img = Image.open(img_path).convert('RGB')
  #---------

  #resize
  img = ImageOps.contain(img, (224,224), Image.BICUBIC)
  #---------

  #add pading
  img = ImageOps.pad(img, (264, 264), centering=(0, 0))
  #---------

  #resize
  img = ImageOps.contain(img, (224,224))
  #---------

  #save
  img.save(img_path)
  #---------

In [ ]:
#train data
for img_path in tqdm(lis):
  img = resize_pad(img_path)

100%|██████████| 682/682 [00:12<00:00, 54.19it/s]


In [ ]:
#test data
lis = glob('/content/test/images/*.jpg')
for img_path in tqdm(lis):
  img = resize_pad(img_path)

100%|██████████| 1120/1120 [00:08<00:00, 136.20it/s]


## Model

### Collect all image to list

In [ ]:
def append_img(lis):
  #collect image
  img_list = []

  #loop
  for img_path in tqdm(lis):
    img = Image.open(img_path).convert("RGB")
    img_list.append(img)
  
  return img_list

In [ ]:
#---queries---
queries = []
path_queries = glob('/content/queries/queries/**/*.jpg')
path_queries = natsorted(path_queries)
queries = append_img(path_queries)
#---------------

100%|██████████| 682/682 [00:00<00:00, 1250.46it/s]


In [ ]:
#----test----
base = '/content/test/images'
sample = pd.read_csv('/content/sample_submission.csv')

path_test = []
for img_path in sample['img_file']:
  img_path = os.path.join(base, img_path)
  path_test.append(img_path)


queries_t = append_img(path_test)
#---------------

100%|██████████| 1120/1120 [00:00<00:00, 1291.30it/s]


### Craete Model

In [ ]:
#SentenceTransformer
img_model = SentenceTransformer('clip-ViT-B-32')

#extract feature
embedded_queries = img_model.encode(queries)
embedded_queries = np.array(embedded_queries)

embedded_queries_t = img_model.encode(queries_t)
embedded_queries_t = np.array(embedded_queries_t)

In [ ]:
annoy_index = AnnoyIndex(512, 'manhattan')  

# add features to annoy_index
for i in tqdm(range(len(embedded_queries))):
    feature = embedded_queries[i]
    # Adds each feature vector to annoy index
    annoy_index.add_item(i, feature)

# Builds 99 search trees for the items added to index
annoy_index.build(99)

100%|██████████| 682/682 [00:00<00:00, 24299.73it/s]


True

### Predict

In [ ]:
id2label = glob('/content/queries/queries/**/*.jpg')
id2label = natsorted(id2label)
id2label = [os.path.basename(os.path.dirname(i)) for i in id2label]

In [ ]:
pred = []
for i in tqdm(embedded_queries_t):
  #annoy
  a = annoy_index.get_nns_by_vector(i, len(id2label), include_distances=True)
  thresh = a[1][0]

  # ----check thresh----
  if thresh < 114.38000000000378:

    # ---select top 5---
    ensemble = []
    for i in range(5):
      ensemble.append(id2label[a[0][i]])

    cls = stats.mode(ensemble, keepdims=True)[0][0]
    pred.append(int(cls))
    # ---------------------

  else:
    pred.append(22)
  # ----------------------


  0%|          | 0/1120 [00:00<?, ?it/s]<ipython-input-111-e520e1d820f8>:15: DeprecationWarning: Support for non-numeric arrays has been deprecated as of SciPy 1.9.0 and will be removed in 1.11.0. `pandas.DataFrame.mode` can be used instead, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mode.html.
  cls = stats.mode(ensemble, keepdims=True)[0][0]
100%|██████████| 1120/1120 [00:03<00:00, 347.61it/s]


In [ ]:
sample = pd.read_csv('/content/sample_submission.csv')
sample['class'] = pred
sample.to_csv('kaggle_machima.csv')
sample.head()

,img_file,class
0,64ccfecf-e451-49a8-aa3f-acf2622a9a5c.jpg,12
1,c6df1385-382a-4428-b41e-f2d729b90c87.jpg,22
2,af30e9d0-da6e-42bd-814e-c70a0c16e554.jpg,22
3,3fc8998e-0324-426c-8233-6b76abc7e200.jpg,22
4,309d1085-0d11-4411-9555-b24cc8fcee02.jpg,22


In [ ]:
#public
nu = pd.read_csv('/content/nu.csv')
nu_class = nu['class'].tolist()
pub = f1_score(nu_class[:560], pred[:560], average='macro')
print(f'public score {pub}')

public score 0.9570677451971689


In [ ]:
#private
private = f1_score(nu_class[560:], pred[560:], average='macro')
print(f'private score {private}')

private score 0.8419628556623328


In [ ]:
#all
all_score = f1_score(nu_class, pred, average='macro')
print(f'all score {all_score}')

all score 0.9665805829378549
